In [1]:
import os
os.environ["JAVA_HOME"] = "/mnt/extproj/projekte/textmining/markus_tm_projs/jdk/openlogic-openjdk-11.0.22+7-linux-x64"

MXPLORE_PATH="/mnt/extproj/projekte/textmining/mxplore/"

from collections import Counter

In [2]:
from collections import defaultdict
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import Window
import glob

In [3]:
if not "spark" in globals() or spark is None:
    print("Creating builder")
    spark = SparkSession.builder.\
            config("spark.executor.memory", "70g").\
            config("spark.driver.memory", "50g").\
            config("spark.memory.offHeap.enabled",True).\
            config("spark.memory.offHeap.size","16g").\
            config("spark.sql.shuffle.partitions", 300).\
            appName('mirexplore').getOrCreate()
    
#spark.conf.set("spark.executor.memory", "70g")
#spark.conf.set("spark.driver.memory", "50g")
#spark.conf.set("spark.memory.offHeap.enabled",True)
#spark.conf.set("spark.memory.offHeap.size","16g")  
#spark.conf.set("spark.sql.shuffle.partitions", 300)

Creating builder


/mnt/extproj/projekte/bartelt/software/miniconda3/envs/mirexplore/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
df_int = spark.read.parquet(MXPLORE_PATH+"/mxresults/mx_mirna_gene_consensus_parquet/")
df_int = df_int.withColumn("doc_id", explode("evidence_documents"))

In [5]:
relDocIDs = set([x["doc_id"] for x in df_int.select("doc_id").collect()])
print(len([x for x in relDocIDs if x.startswith("PMC")]))

55331


In [6]:
def read_date_file(inputpath, relevantDocIDs):
    db_entries = list()
    ignoredConcepts = Counter()


    for infile in glob.glob("{}/*.journal".format(inputpath)):
        with open(infile, "r") as fin:
            
            for iline, line in enumerate(fin):
                aline = line.split("\t")
                aline = [x.strip() for x in aline]

                docID = aline[0]
                
                if not docID in relevantDocIDs:
                    continue
                
                if len(aline) < 2:
                    continue
                
                long_journal = aline[1]
                short_journal = None
                
                if len(aline) >= 3 and len(aline[2]) > 0:
                    short_journal = aline[2]
                                
                journal = short_journal if not short_journal is None else long_journal
                db_entries.append( (docID, journal) )
                
    return db_entries


In [7]:
docAuthors = list(set(read_date_file("/mnt/extproj/projekte/textmining/pubmed_feb24/", relDocIDs)))
print(len(docAuthors))

47280


In [8]:
docAuthorsPMC = list(set(read_date_file("/mnt/extproj/projekte/textmining/pmc_feb24/oa_comm/", relDocIDs)))
print(len(docAuthorsPMC))

55331


In [9]:
df = spark.createDataFrame(docAuthors+docAuthorsPMC, ["doc_id", "journal"])

In [10]:
df.show()

+--------+--------------------+
|  doc_id|             journal|
+--------+--------------------+
|25213330|         Horm Cancer|
|32781357| Biomed Pharmacother|
|38030294|         Int Heart J|
|22541436|                Cell|
|26503214|         Tumour Biol|
|22580605|            Oncogene|
|31832755|     Int J Legal Med|
|31476285|         Am J Pathol|
|30479162|       Hum Gene Ther|
|33116855|    Cancer Manag Res|
|32053576|       Med Sci Monit|
|33413548|  Stem Cell Res Ther|
|33500625|   Onco Targets Ther|
|33279663|        Exp Gerontol|
|28627449|     Mol Cell Probes|
|33621954|   Aging (Albany NY)|
|34424826|       Bioengineered|
|33275250|Eur Rev Med Pharm...|
|29802737|        Mol Carcinog|
|32372215|J Egypt Natl Canc...|
+--------+--------------------+
only showing top 20 rows



In [11]:
df.write.parquet(MXPLORE_PATH+"/mxresults/mx_document_journal", mode="overwrite")

In [12]:
df.orderBy("doc_id").show()

+--------+--------------------+
|  doc_id|             journal|
+--------+--------------------+
|10760272|Proc Natl Acad Sc...|
|12576545|Proc Natl Acad Sc...|
|12812784|            Dev Biol|
|15003116|         Genome Biol|
|15361871|           Nat Genet|
|15504739|         J Biol Chem|
|15538371|              Nature|
|15634332|              FEBS J|
|15648093|      J Cell Physiol|
|15738415|Proc Natl Acad Sc...|
|15766526|                Cell|
|15944709|              Nature|
|16041695|            J Pathol|
|16096373|          Cell Cycle|
|16151463|            Leukemia|
|16166262|Proc Natl Acad Sc...|
|16227573|       Mol Cell Biol|
|16235244|Genes Chromosomes...|
|16249029|            Leuk Res|
|16266980|          Cancer Res|
+--------+--------------------+
only showing top 20 rows

